In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full = X_train_full.reshape((60000, 28,28,1))
X_train_full = X_train_full.astype('float32') / 255

X_test = X_test.reshape((10000, 28, 28,1))
X_test = X_test.astype('float32') / 255

In [ ]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:] 
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
def create_model():
  model = keras.models.Sequential(
    [                              
      keras.layers.Conv2D(64, (5,5), padding="same", input_shape=(28,28,1)),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
      
      keras.layers.Conv2D(128, (3,3), padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.Conv2D(64, (1,1), padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.Conv2D(128, (3,3), padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.MaxPooling2D((2,2)),

      keras.layers.Conv2D(256, (3,3), padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.Conv2D(128, (1,1), padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.Conv2D(256, (3,3), padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.MaxPooling2D((2,2)),

      keras.layers.Flatten(),
      keras.layers.Dense(256),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.Dropout(0.6),

      keras.layers.Dense(128),
      keras.layers.LeakyReLU(alpha=0.1),
      keras.layers.Dropout(0.6),

      keras.layers.Dense(10, activation="softmax")
    ]                        
  )
  return model


In [ ]:
model = create_model()
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=15,
                    batch_size=32,
                    validation_data=(X_valid, y_valid)) 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
#plt.gca().set_ylim(0,1)
#plt.savefig('CNN_1_15_epok.png')

In [ ]:
test_loss, test_acc = model.evaluate(X_test.astype(np.float32), y_test.astype(np.float32))

In [ ]:
test_acc

In [ ]:
#keras.utils.plot_model(model, "CNN_1.png", show_shapes=True)

In [ ]:
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
predictions = model.predict(X_test)
predictions = tf.argmax(predictions, axis=-1)
cm = tf.math.confusion_matrix(y_test, predictions)
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

In [ ]:
sns.heatmap(
    cm, annot = True,
    xticklabels=labels,
    yticklabels=labels)
plt.tight_layout()
plt.xlabel('Predicted')
plt.ylabel('True')
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)
# when saving, specify the DPI
#plt.savefig("CNN_new_heatmap.png", dpi = 100)